In [1]:
# ------------------------------------------------------------
# Imports
# ------------------------------------------------------------
import os
import glob
import shutil
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.signal as sg
from sklearn.preprocessing import LabelEncoder
import kagglehub

# ------------------------------------------------------------
# Dataset setup
# ------------------------------------------------------------
dataset_name = "noisy-drone-rf-signal-classification-v2"

kaggle_cache_root = f"/scratch/rameyjm7/kaggle"
os.makedirs(kaggle_cache_root, exist_ok=True)
os.environ["KAGGLEHUB_CACHE"] = kaggle_cache_root

scratch_root = f"/scratch/rameyjm7/{dataset_name}"
os.makedirs(scratch_root, exist_ok=True)

# ------------------------------------------------------------
# Kaggle Download
# ------------------------------------------------------------
kaggle_path = kagglehub.dataset_download("sgluege/noisy-drone-rf-signal-classification-v2")
print("Kaggle dataset path:", kaggle_path)

# Copy to scratch
if len(os.listdir(scratch_root)) == 0:
    print("Copying dataset into scratch...")
    shutil.copytree(kaggle_path, scratch_root, dirs_exist_ok=True)
else:
    print("Scratch directory already populated.")

print("Scratch dataset location:", scratch_root)
print("Scratch contents:", os.listdir(scratch_root))

# ------------------------------------------------------------
# Load .pt IQ Data
# ------------------------------------------------------------
# ------------------------------------------------------------
# Load .pt IQ Data  (FIXED)
# ------------------------------------------------------------
drone_root = os.path.join(scratch_root, "drone_RF_data")

pt_files = glob.glob(os.path.join(drone_root, "*.pt"))
print("Found .pt IQ files:", len(pt_files))
if len(pt_files) == 0:
    raise RuntimeError("No .pt IQ files found. Check dataset structure.")

signals = []
labels = []
snrs = []

for f in pt_files:
    data = torch.load(f)

    # Correct keys:
    # x_iq shape = (2, N) = [I_row, Q_row]
    x = data["x_iq"].numpy()
    iq = x.T   # transpose -> shape (N, 2)
    signals.append(iq)

    labels.append(int(data["y"].item()))
    snrs.append(float(data["snr"].item()))

signals = np.array(signals, dtype=object)  # variable-length sequences allowed
labels = np.array(labels)
snrs = np.array(snrs)

print("Signals loaded:", len(signals))
print("Example shape:", signals[0].shape)
print("Label set:", np.unique(labels))
print("SNR range:", snrs.min(), "to", snrs.max())


# ------------------------------------------------------------
# Encode labels to class names (0..5 for 6 drones)
# ------------------------------------------------------------
enc = LabelEncoder()
y = enc.fit_transform(labels)
class_names = list(enc.classes_)
print("Encoded classes:", class_names)

# ------------------------------------------------------------
# EDA: Single sample I/Q + Spectrum + Waterfall
# ------------------------------------------------------------
idx = 0
iq = signals[idx]
I = iq[:,0]
Q = iq[:,1]
complex_sig = I + 1j * Q

fs = 1e6  # placeholder

fig, ax = plt.subplots(3,1, figsize=(12,12))

# 1. Time Domain
ax[0].plot(I, label="I")
ax[0].plot(Q, label="Q")
ax[0].set_title(f"Time Domain I/Q (sample {idx}, class={labels[idx]}, snr={snrs[idx]})")
ax[0].legend()

# 2. Spectrum
f, Pxx = sg.welch(complex_sig, fs=fs, nperseg=1024)
ax[1].semilogy(f, Pxx)
ax[1].set_title("Magnitude Spectrum")

# 3. Waterfall
f_s, t_s, Sxx = sg.spectrogram(
    complex_sig,
    fs=fs,
    nperseg=256,
    noverlap=128
)
pcm = ax[2].pcolormesh(t_s, f_s, 10*np.log10(Sxx + 1e-12), shading="auto")
ax[2].set_title("Waterfall Spectrogram")
ax[2].set_ylabel("Freq (Hz)")
ax[2].set_xlabel("Time (sec)")
plt.colorbar(pcm, ax=ax[2])

plt.tight_layout()
plt.show()

# ------------------------------------------------------------
# MULTI-SAMPLE SPECTROGRAM GRID (like RadDet)
# ------------------------------------------------------------
num_plots = min(9, len(signals))
rows = 3
cols = 3

fig, axes = plt.subplots(rows, cols, figsize=(14,12))

for i in range(num_plots):
    ax = axes[i//cols][i%cols]

    I = signals[i][:,0]
    Q = signals[i][:,1]
    complex_sig = I + 1j * Q

    f_s, t_s, Sxx = sg.spectrogram(complex_sig, fs=fs, nperseg=256, noverlap=128)
    ax.pcolormesh(t_s, f_s, 10*np.log10(Sxx + 1e-12), shading="auto")

    ax.set_title(f"class={labels[i]} snr={snrs[i]}")
    ax.set_xticks([])
    ax.set_yticks([])

plt.suptitle("Spectrogram Grid of Drone RF IQ Samples", y=1.02, fontsize=14)
plt.tight_layout()
plt.show()


Kaggle dataset path: /scratch/rameyjm7/kaggle/datasets/sgluege/noisy-drone-rf-signal-classification-v2/versions/1
Scratch directory already populated.
Scratch dataset location: /scratch/rameyjm7/noisy-drone-rf-signal-classification-v2
Scratch contents: ['drone_RF_data']
Found .pt IQ files: 17744


: 

In [3]:
import torch

sample = "/scratch/rameyjm7/noisy-drone-rf-signal-classification-v2/drone_RF_data/IQdata_sample0_target0_snr-14.pt"
data = torch.load(sample)

print("Type:", type(data))
print("Keys:", data.keys() if hasattr(data, "keys") else "Not a dict")
print(data)


Type: <class 'dict'>
Keys: dict_keys(['x_iq', 'y', 'snr'])
{'x_iq': tensor([[ 8.5068e-04, -3.7270e-03, -1.7382e-03,  ...,  9.1976e-01,
          3.2465e+00, -7.9968e-01],
        [-1.4233e-03,  4.8080e-03,  6.0526e-03,  ..., -5.9499e-01,
          1.2086e+00,  2.1661e+00]]), 'y': tensor(0), 'snr': tensor(-14)}


In [1]:
# ------------------------------------------------------------
# Imports
# ------------------------------------------------------------
import os
import glob
import shutil
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.signal as sg
from sklearn.preprocessing import LabelEncoder
import kagglehub

# ------------------------------------------------------------
# Dataset setup
# ------------------------------------------------------------
dataset_name = "noisy-drone-rf-signal-classification-v2"

kaggle_cache_root = f"/scratch/rameyjm7/kaggle"
os.makedirs(kaggle_cache_root, exist_ok=True)
os.environ["KAGGLEHUB_CACHE"] = kaggle_cache_root

scratch_root = f"/scratch/rameyjm7/{dataset_name}"
os.makedirs(scratch_root, exist_ok=True)

# ------------------------------------------------------------
# Kaggle Download
# ------------------------------------------------------------
kaggle_path = kagglehub.dataset_download("sgluege/noisy-drone-rf-signal-classification-v2")
print("Kaggle dataset path:", kaggle_path)

# ------------------------------------------------------------
# Copy into scratch if empty
# ------------------------------------------------------------
if len(os.listdir(scratch_root)) == 0:
    print("Copying dataset into scratch...")
    shutil.copytree(kaggle_path, scratch_root, dirs_exist_ok=True)
else:
    print("Scratch directory already populated.")

print("Scratch dataset location:", scratch_root)
print("Scratch contents:", os.listdir(scratch_root))

# ------------------------------------------------------------
# Load .pt IQ Data
# ------------------------------------------------------------
drone_root = os.path.join(scratch_root, "drone_RF_data")

pt_files = glob.glob(os.path.join(drone_root, "*.pt"))
print("Found .pt IQ files:", len(pt_files))

signals = []
labels = []
snrs = []

for f in pt_files:
    entry = torch.load(f)

    # Tensor shape is (2, N), we transpose to (N, 2)
    raw = entry["x_iq"].numpy()
    iq = raw.T    # Now shape = (N, 2)
    signals.append(iq)

    labels.append(int(entry["y"].item()))
    snrs.append(float(entry["snr"].item()))

signals = np.array(signals, dtype=object)  # variable-length allowed
labels = np.array(labels)
snrs = np.array(snrs)

print("Loaded signals:", len(signals))
print("Example shape:", signals[0].shape)
print("Labels:", np.unique(labels))
print("SNR range:", snrs.min(), "to", snrs.max())

# ------------------------------------------------------------
# Encode labels (6 drones)
# ------------------------------------------------------------
enc = LabelEncoder()
y = enc.fit_transform(labels)
class_names = list(enc.classes_)
print("Encoded classes:", class_names)

# ------------------------------------------------------------
# EDA: Single sample
# ------------------------------------------------------------
idx = 0
iq = signals[idx]
I = iq[:,0]
Q = iq[:,1]
complex_sig = I + 1j * Q

fs = 1e6  # placeholder

fig, ax = plt.subplots(3,1, figsize=(12,12))

# 1. Time Domain
ax[0].plot(I, label="I")
ax[0].plot(Q, label="Q")
ax[0].set_title(f"Time Domain I/Q (sample {idx}, class={labels[idx]}, snr={snrs[idx]})")
ax[0].legend()

# 2. Spectrum
f, Pxx = sg.welch(complex_sig, fs=fs, nperseg=1024)
ax[1].semilogy(f, Pxx)
ax[1].set_title("Magnitude Spectrum")

# 3. Waterfall
f_s, t_s, Sxx = sg.spectrogram(complex_sig, fs=fs, nperseg=256, noverlap=128)
pcm = ax[2].pcolormesh(t_s, f_s, 10*np.log10(Sxx + 1e-12), shading="auto")
ax[2].set_title("Waterfall Spectrogram")
ax[2].set_ylabel("Freq (Hz)")
ax[2].set_xlabel("Time (sec)")
plt.colorbar(pcm, ax=ax[2])

plt.tight_layout()
plt.show()

# ------------------------------------------------------------
# MULTI-SAMPLE SPECTROGRAM GRID (like RadDet)
# ------------------------------------------------------------
num_plots = min(9, len(signals))
rows = 3
cols = 3

fig, axes = plt.subplots(rows, cols, figsize=(14,12))

for i in range(num_plots):
    ax = axes[i//cols][i%cols]

    iq = signals[i]
    I = iq[:,0]
    Q = iq[:,1]
    csig = I + 1j * Q

    f_s, t_s, Sxx = sg.spectrogram(csig, fs=fs, nperseg=256, noverlap=128)
    ax.pcolormesh(t_s, f_s, 10*np.log10(Sxx + 1e-12), shading="auto")

    ax.set_title(f"class={labels[i]}, snr={snrs[i]}")
    ax.set_xticks([])
    ax.set_yticks([])

plt.suptitle("Spectrogram Grid of Drone RF IQ Samples", y=1.02, fontsize=14)
plt.tight_layout()
plt.show()


Kaggle dataset path: /scratch/rameyjm7/kaggle/datasets/sgluege/noisy-drone-rf-signal-classification-v2/versions/1
Scratch directory already populated.
Scratch dataset location: /scratch/rameyjm7/noisy-drone-rf-signal-classification-v2
Scratch contents: ['drone_RF_data']
Found .pt IQ files: 17744


: 